<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/hojichar_basedon_tsuji_san_kohmoto_sancode_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## コーパスの名前、電話番号、メールアドレスをマスキングする
- ただし名前は、後ろに敬称（さん、くん、君、様）が付くものに限る

In [1]:
!pip install -U hojichar==0.9.0
!pip show hojichar
!pip install regex


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.8 MB/s eta 0:00:00
Name: hojichar
Version: 0.9.0
Summary: Text preprocessing management system.
Home-page: https://github.com/HojiChar/HojiChar
Author: kenta.shinzato
Author-email: hoppiece@gmail.com
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: mmh3, numpy, tqdm
Required-by: 


In [2]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.9 MB/s eta 0:00:00


In [ ]:
import regex as re
import json
import random
import hojichar
from multiprocessing import Pool, cpu_count
from faker import Faker

fake = Faker('ja_JP')  # 日本語のダミーデータを生成するためのFakerインスタンス

num_samples = 100000
dummy_texts = [
    "こんにちは田中君、これはテストデータです。電話番号は08098765432です。佐藤さんのメールアドレスはdef@example.comです。",
    "山田くんのメールアドレスはjtwmdagp@gmail.comで、電話番号は03-4499-3322です。",
    "HojiCharを使用して前処理を行います。村上様の電話番号は、075-4329-9899です。",
]

def replace_entities(text):
    # 名前を置換
    text = re.sub(r'(?P<name>\p{Script=Han}+)(?P<honorific>さん|くん|様|君)',
                  lambda m: fake.last_name() + m.group('honorific'), text)

    # 電話番号を置換
    text = re.sub(r'\d{2,4}-?\d{2,4}-?\d{2,4}', lambda x: fake.phone_number(), text)

    # メールアドレスを置換
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', fake.email(), text)

    return text

def process_text(text):
    replaced_text = replace_entities(text)
    # HojiCharの処理
    cleaner = hojichar.Compose([
        hojichar.document_filters.DocumentNormalizer(),
        hojichar.document_filters.JSONDumper(),
    ])
    document = hojichar.Document(json.dumps({"text": replaced_text}))
    cleaned_text = cleaner.apply(document).text
    return cleaned_text

def main():
    random_texts = [random.choice(dummy_texts) for _ in range(num_samples)]
    num_cores = cpu_count()

    with Pool(num_cores) as pool:
        processed_texts = pool.map(process_text, random_texts)

    with open("your_text.jsonl", "w") as f:
        for processed_text in processed_texts:
            f.write(processed_text + "\n")

    # 結果の表示
    num_display = 100
    with open("your_text.jsonl", "r") as f:
        for i, line in enumerate(f):
            if i >= num_display:
                break
            data = json.loads(line)
            actual_text = json.loads(data['text'])
            print(actual_text)

if __name__ == "__main__":
    main()

## HFdatasetsから、日本語データセットのロード
- ログイン（アクセストークン）要

In [6]:
# パッケージのインストール
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00


In [13]:
!pip install huggingface_hub
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

# データセットの読み込み
from datasets import load_dataset
dataset = load_dataset("uonlp/CulturaX","ja",use_auth_token=True)

# データセットの確認
dataset

Resolving data files:   0%|          | 0/160 [00:00<?, ?it/s]

## HGdatasetsのフィルター処理